In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Data augmentation for training data
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Data preparation for test data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

def conv_layer(in_channels, out_channels, kernel_size=3, padding=1):
    """ Convolutional layer setup with Batch Normalization
    """
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding), 
              nn.BatchNorm2d(out_channels)]
    return nn.Sequential(*layers)


class ResidualBlock(nn.Module):
  """ Base class for residual block. Contains two convolutional layers with
      parameters based on initialization values. Uses ReLU as activation
      function for each convolutional layer
  """
  def __init__(self, in_channels, out_channels, kernel_size, padding):
    super(ResidualBlock, self).__init__()

    self.conv1 = conv_layer(in_channels, out_channels, kernel_size, padding)
    self.conv2 = conv_layer(out_channels, out_channels, kernel_size, padding)
  
  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out = F.relu(out + x)
    return out

class CNNModel(nn.Module):
    """ CNN model for CIFAR-10 dataset. Contains four general convolutional layers,
        2 residual blocks with two convolutional layers each and 2 fully connected
        layers. Uses ReLU as an activation function for each layer except the output
        layer. Implements dropout after each residual block and after first fully
        connected layer.
    """
    def __init__(self, in_channels, out_classes):
        super(CNNModel, self).__init__()
        
        # self.conv = conv_layer(in_channels, 32, kernel_size=3, padding=1)
        self.conv1 = conv_layer(in_channels, 64, kernel_size=3, padding=1)
        self.conv2 = conv_layer(64, 128, kernel_size=3, padding=1)
        self.res_block_1 = ResidualBlock(128, 128, kernel_size=3, padding=1)
        self.conv3 = conv_layer(128, 256, kernel_size=5, padding=1)
        self.conv4 = conv_layer(256, 512, kernel_size=5, padding=1)
        self.res_block_2 = ResidualBlock(512, 512, kernel_size=3, padding=1)
        self.fully_connected_1 = nn.Linear(512, 512)
        self.fully_connected_2 = nn.Linear(512, out_classes)
        
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, kernel_size=2)
        out = self.res_block_1(out)
        out = F.dropout(out, 0.1)

        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, kernel_size=2)
        out = self.res_block_2(out)

        out = F.max_pool2d(out, kernel_size=4)
        out = out.view(out.size(0), -1)
        out = F.dropout(out, 0.1)

        out = F.relu(self.fully_connected_1(out))
        out = F.dropout(out, 0.1)
        out = self.fully_connected_2(out)
        out = F.log_softmax(out, dim=1)
        return out

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    """ Train the model with training dataset
    """
    model.train()
    count = 0
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
      #data is the image
      #target is the ground truth
        data, target = data.to(device), target.to(device)

        # clear derivatives
        optimizer.zero_grad()

        # forward propagation
        output = model(data)

        # data loss
        loss=F.cross_entropy(output, target)
        running_loss += loss.item()

        # backpropagation
        loss.backward()

        # updating parameters
        optimizer.step()
        count +=1

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    running_loss /= count
    train_losses.append(running_loss)

def test( model, device, test_loader):
    """ Test model with test dataset
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
start_time = time.time()
train_losses = []
test_losses = []

# parameters
batch_size = 128
epochs = 50
lr = 0.005
torch.manual_seed(100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

model = CNNModel(3,10).to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)

end_time = time.time() 
print ('Training and Testing total excution time: %s seconds ' % (end_time-start_time))

Files already downloaded and verified
Files already downloaded and verified
Train Epoch: 1 [0/50000 (0%)]	Loss: 2.356320
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.558804
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.277456
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.969870

Test set: Average loss: 1.1563, Accuracy: 5973/10000 (59.73%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.950347
Train Epoch: 2 [12800/50000 (26%)]	Loss: 0.893816
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.888234
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.796646

Test set: Average loss: 0.7022, Accuracy: 7571/10000 (75.71%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.608334
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.581090
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.541661
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.659908

Test set: Average loss: 0.6263, Accuracy: 7826/10000 (78.26%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.589508
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.631230
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.618668
Tr